# Packages and Configurations

In [1]:
import astro_util, os, re
import numpy as np
from astropy.io import fits

# Make Dark Frame Master

In [7]:
dark_data, dark_headers = astro_util.get_all_data_with_exposure(
    5.0, path="./data/unprocessed/dark/")

In [8]:
for dark_name in os.listdir("./data/unprocessed/dark/"):
    split_name = dark_name.split('_')
    if "5.0s" in split_name:
        dark_name = split_name
        break
for (i, substr) in enumerate(dark_name):
    if bool(re.fullmatch("[0-9]+\.?[0-9]*s", substr)):
        dark_name[i] = str(len(dark_headers)) + 'x' + substr
    if bool(re.fullmatch("[0-9]+T[0-9]+\.fits", substr)):
        dark_name[i] = substr.split('T')[0] + ".fits"
    if bool(re.fullmatch("[0-9]+T[0-9]+", substr)):
        dark_name[i] = substr.split('T')[0]
print(dark_name)

['Shamloo', 'Yeganeh', 'Khanali', 'Dark', '15x5.0s', 'ISO1600', '1200D', '20230719.fits']


In [9]:
fits.PrimaryHDU(data=np.array(np.median(dark_data, axis=0), dtype=np.uint16),
    header=astro_util.build_master_header(dark_headers[0], len(dark_headers))
    ).writeto("./data/master/" + '_'.join(dark_name), overwrite=True)

c:\Users\SLHHe\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [10]:
dark_data = None

# Make Dark-Corrected Flat Frame

In [7]:
flat_data, flat_headers = astro_util.get_all_data_with_exposure(
    2.5, path="./data/unprocessed/flat/")

In [8]:
for filename in os.listdir("./data/unprocessed/flat/"):
    split_name = filename.split('_')
    if "2.5s" in split_name:
        flat_name = split_name
        break
for (i, substr) in enumerate(flat_name):
    if bool(re.fullmatch("[0-9]+\.?[0-9]*s", substr)):
        flat_name[i] = str(len(flat_headers)) + 'x' + substr
    if bool(re.fullmatch("[0-9]+T[0-9]+\.fits", substr)):
        flat_name[i] = substr.split('T')[0] + ".fits"
    if bool(re.fullmatch("[0-9]+T[0-9]+", substr)):
        flat_name[i] = substr.split('T')[0]
print(flat_name)

['Shamloo', 'Yeganeh', 'Khanali', 'Flat', '19x2.5s', 'ISO100', '1200D', 'Newton8', '20230718', 'Aznaveh.fits']


In [10]:
for filename in os.listdir("./data/master/"):
    split_name = filename.split('_')
    if "Dark" in split_name:
        for substr in split_name:
            if substr.endswith("2.5s"):
                flat_dark = fits.open("./data/master/" + filename)[0].data
flat_data = np.median(flat_data, axis=0) - flat_dark
flat_data[flat_data < 0] = 0
flat_data = flat_data.astype(np.uint16)
flat_header = astro_util.build_master_header(flat_headers[0], len(flat_headers))

c:\Users\SLHHe\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:758: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [11]:
fits.PrimaryHDU(data=flat_data, header=flat_header).writeto(
    "./data/master/" + '_'.join(flat_name), overwrite=True)

In [12]:
flat_data = None

# Flat-Field and Dark Current Correction

In [14]:
for filename in os.listdir("./data/master/"):
    substrs = filename.split('_')
    if "Dark" in substrs:
        for substr in substrs:
            if substr.endswith("1.6s"):
                dark_master = fits.open("./data/master/" + filename)
                break

In [15]:
for filename in os.listdir("./data/master/"):
    substrs = filename.split('_')
    if "Flat" in substrs:
        for substr in substrs:
            flat_master = fits.open("./data/master/" + filename)
            break

In [19]:
gain_table = flat_master[0].data / np.median(flat_master[0].data)
gain_table[gain_table == 0] = 1

In [26]:
for filename in os.listdir("./data/unprocessed/stars/"):
    fitsfile = fits.open("./data/unprocessed/stars/" + filename)
    header = fitsfile[0].header
    header["CALSTAT"] = 'F'
    data = fitsfile[0].data
    data = (data.astype(np.float64) - dark_master[0].data) / gain_table
    data[data < 0] = 0
    data = data.astype(np.uint16)
    fits.PrimaryHDU(data=data, header=header).writeto(
        "./data/stars/" + filename, overwrite=True)